In [ ]:
import json
import os
from pathlib import Path

import pandas as pd

hex_colours = ["#e66581", "#5799c9", "#f5a252"]

In [ ]:
# Set filepaths
ABSOLUTE_HERE = Path(os.getcwd()).parent
processed_data_dir = ABSOLUTE_HERE.parent.joinpath("data/processed")
data_filepath = processed_data_dir.joinpath("survey-responses.csv")
questions_filepath = processed_data_dir.joinpath("questions_metadata.json")

In [ ]:
# Load datasets
with open(questions_filepath) as stream:
    question_metadata = json.load(stream)

data = pd.read_csv(data_filepath, index_col="Voter")

### Question 1

In [ ]:
question = question_metadata[0]["question"]
choices = question_metadata[0]["choices"]

hist = data[question].value_counts()

for choice in choices:
    if choice not in hist.index:
        hist = pd.concat([hist, pd.Series(0, index=[choice])])

hist.plot(kind="bar")